http://global.oup.com/uk/orc/biosciences/exp_design/holmes3e/student/walkthroughs/7843589/srhr/
https://rdrr.io/cran/rcompanion/man/scheirerRayHare.html
http://www.sthda.com/english/wiki/two-way-anova-test-in-r
http://www.gribblelab.org/stats/notes/TwowayANOVA.pdf

In [1]:
library(repr)

# Muda o tamanho plot size to 8 x 4
options(repr.plot.width=8, repr.plot.height=4)
curve(sin(x), from = 0, to=4*pi, n = 200)

In [2]:
dados <- ToothGrowth

In [3]:
str(dados)

'data.frame':	60 obs. of  3 variables:
 $ len : num  4.2 11.5 7.3 5.8 6.4 10 11.2 11.2 5.2 7 ...
 $ supp: Factor w/ 2 levels "OJ","VC": 2 2 2 2 2 2 2 2 2 2 ...
 $ dose: num  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 ...


In [4]:
dados$dose <- factor(dados$dose)

In [5]:
head(dados)

len,supp,dose
4.2,VC,0.5
11.5,VC,0.5
7.3,VC,0.5
5.8,VC,0.5
6.4,VC,0.5
10.0,VC,0.5


In [6]:
table(dados$supp,dados$dose)

    
     0.5  1  2
  OJ  10 10 10
  VC  10 10 10

In [7]:
str(dados)

'data.frame':	60 obs. of  3 variables:
 $ len : num  4.2 11.5 7.3 5.8 6.4 10 11.2 11.2 5.2 7 ...
 $ supp: Factor w/ 2 levels "OJ","VC": 2 2 2 2 2 2 2 2 2 2 ...
 $ dose: Factor w/ 3 levels "0.5","1","2": 1 1 1 1 1 1 1 1 1 1 ...


# Anova de dois fatores paramÃ©trica

In [8]:
library(car)

In [10]:
res <- aov(len ~ supp + dose + supp*dose, data = dados)
summary(res)

            Df Sum Sq Mean Sq F value   Pr(>F)    
supp         1  205.4   205.4  15.572 0.000231 ***
dose         2 2426.4  1213.2  92.000  < 2e-16 ***
supp:dose    2  108.3    54.2   4.107 0.021860 *  
Residuals   54  712.1    13.2                     
---
Signif. codes:  0 â***â 0.001 â**â 0.01 â*â 0.05 â.â 0.1 â â 1

In [11]:
Anova(res, type = "III") # Para dados nÃ£o balanceados 

,Sum Sq,Df,F value,Pr(>F)
(Intercept),1750.3290,1,132.729911,3.602548e-16
supp,137.8125,1,10.450516,2.092470e-03
dose,885.2647,2,33.565433,3.363190e-10
supp:dose,108.3190,2,4.106991,2.186027e-02
Residuals,712.1060,54,NA,NA


In [12]:
install.packages("ggpubr")
library(ggpubr)

Installing package into â/home/nbuser/Râ
(as âlibâ is unspecified)
also installing the dependencies âggrepelâ, âggsciâ, âcowplotâ, âggsignifâ

Loading required package: ggplot2
Loading required package: magrittr


In [13]:
# Box-plots
ggboxplot(dados, x = "dose", y = "len", color = "supp",
          palette = c("#00AFBB", "#E7B800"))

In [15]:
# Barras de erros com linhas
ggline(dados, x = "dose", y = "len", color = "supp",
       add = c("mean_se", "dotplot"),
       palette = c("#00AFBB", "#E7B800"))

`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


## Post hoc

In [16]:
TukeyHSD(res, which = "supp")

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = len ~ supp + dose + supp * dose, data = dados)

$supp
      diff       lwr       upr     p adj
VC-OJ -3.7 -5.579828 -1.820172 0.0002312


In [17]:
TukeyHSD(res, which = "dose")

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = len ~ supp + dose + supp * dose, data = dados)

$dose
        diff       lwr       upr   p adj
1-0.5  9.130  6.362488 11.897512 0.0e+00
2-0.5 15.495 12.727488 18.262512 0.0e+00
2-1    6.365  3.597488  9.132512 2.7e-06


In [18]:
library(multcomp)

summary(glht(res, linfct = mcp(dose = "Tukey")))

Loading required package: mvtnorm
Loading required package: survival
Loading required package: TH.data
Loading required package: MASS

Attaching package: âTH.dataâ

The following object is masked from âpackage:MASSâ:

    geyser

Warning message in mcp2matrix(model, linfct = linfct):
âcovariate interactions found -- default contrast might be inappropriateâ


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: aov(formula = len ~ supp + dose + supp * dose, data = dados)

Linear Hypotheses:
             Estimate Std. Error t value Pr(>|t|)    
1 - 0.5 == 0    9.470      1.624   5.831   <1e-04 ***
2 - 0.5 == 0   12.830      1.624   7.900   <1e-04 ***
2 - 1 == 0      3.360      1.624   2.069    0.106    
---
Signif. codes:  0 â***â 0.001 â**â 0.01 â*â 0.05 â.â 0.1 â â 1
(Adjusted p values reported -- single-step method)


# Pressupostos

In [20]:
plot(res) # GrÃ£ficos de resÃ­duos, q-q (Normal), resÃ­duos padronizados, resÃ­duos pelo primeiro fator

In [30]:
# Teste para homogeneidade de variÃ¢ncias (homocedasticidade)
# Apenas a interaÃ§Ã£o de nÃ­vel mais alto

leveneTest(len ~ supp*dose,data = dados,location="median")

,Df,F value,Pr(>F)
group,5,1.708578,0.1483606
,54,NA,NA


In [34]:
# Testes de aderÃªncia dos resÃ­duos Ã  Normal
ks.test(resid(res), "pnorm")
shapiro.test(resid(res))

Warning message in ks.test(resid(res), "pnorm"):
âties should not be present for the Kolmogorov-Smirnov testâ


	One-sample Kolmogorov-Smirnov test

data:  resid(res)
D = 0.31653, p-value = 1.201e-05
alternative hypothesis: two-sided



	Shapiro-Wilk normality test

data:  resid(res)
W = 0.98499, p-value = 0.6694


In [33]:
# Teste de Lilliefors (KS para dados com empates)

library(nortest)
lillie.test(resid(res))


	Lilliefors (Kolmogorov-Smirnov) normality test

data:  resid(res)
D = 0.06847, p-value = 0.6913


# Anova de dois fatores nÃ£o-paramÃ©trica

In [36]:
# Teste de Scheirer-Ray-Hare (Sokal)

bellis = c(1, 2.5, 2.5, 4, 
           5, 6.5, 6.5, 9.5,
           8,9.5, 12, 12,
           14.5, 12, 14.5, 16)
location = rep(c("Cricket", "Lawn"), each = 8)
fertiliser = rep(c("no", "yes", "no", "yes"), each = 4)

aov.r = aov(rank(bellis) ~ location*fertiliser)
summary(aov.r)

# Extrai as somas dos quadrados
df = anova(res)[,"Df"]
sum_df = sum(df)
ss = anova(res)[, "Sum Sq"]
sum_ss = sum(ss)

# Calcula as mÃ©dias quadrÃ¡ticas
MS = sum_ss/sum_df

# Calcula a estatÃ­stica (H)
H_supp = ss[1]/MS
H_supp

H_dose = ss[2]/MS
H_dose

H_int = ss[3]/MS
H_int

# Calcula os nÃ­veis de significÃ¢ncia 
1-pchisq(H_supp, df[1])
1-pchisq(H_dose, df[2])
1-pchisq(H_int, df[3])

                    Df Sum Sq Mean Sq F value   Pr(>F)    
location             1 232.56  232.56  79.452 1.22e-06 ***
fertiliser           1  68.06   68.06  23.253 0.000418 ***
location:fertiliser  1   0.25    0.25   0.085 0.775088    
Residuals           12  35.12    2.93                     
---
Signif. codes:  0 â***â 0.001 â**â 0.01 â*â 0.05 â.â 0.1 â â 1

[1] 3.509535

[1] 41.46899

[1] 1.851226

[1] 0.06101657

[1] 9.888349e-10

[1] 0.3962883

In [37]:
# Teste de Van der Waerden (Conover)
res.np <- aov(qqnorm(len,plot=FALSE)$x ~ supp + dose + supp*dose, data = dados)
Anova(res.np, type = "III")

,Sum Sq,Df,F value,Pr(>F)
(Intercept),4.141622,1,13.782433,4.871426e-04
supp,2.586423,1,8.607064,4.906484e-03
dose,12.528808,2,20.846596,1.953635e-07
supp:dose,1.925236,2,3.203386,4.845286e-02
Residuals,16.227005,54,NA,NA


In [31]:
summary(glht(res.np, linfct = mcp(dose = "Tukey")))

Warning message in mcp2matrix(model, linfct = linfct):
âcovariate interactions found -- default contrast might be inappropriateâ


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: aov(formula = qqnorm(len, plot = FALSE)$x ~ supp + dose + supp * 
    dose, data = dados)

Linear Hypotheses:
             Estimate Std. Error t value Pr(>|t|)    
1 - 0.5 == 0   1.0662     0.2452   4.349 0.000174 ***
2 - 0.5 == 0   1.5464     0.2452   6.308  < 1e-04 ***
2 - 1 == 0     0.4801     0.2452   1.958 0.132468    
---
Signif. codes:  0 â***â 0.001 â**â 0.01 â*â 0.05 â.â 0.1 â â 1
(Adjusted p values reported -- single-step method)
